In [37]:

import os, random, math, json
import numpy as np
import pandas as pd
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm

# Notebook / Windows safety
os.environ["TOKENIZERS_PARALLELISM"] = "false"
try:
    torch.multiprocessing.set_start_method("spawn", force=True)
except RuntimeError:
    pass

device = "cuda" if torch.cuda.is_available() else "cpu"
pin = (device == "cuda")
print("Device:", device)


Device: cuda


In [38]:
CFG = dict(
    csv_path="data/encoder_data.csv",
    embed_data_path = "data/data.csv",            
    outdir="data", 
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    epochs=6,
    batch_size=128,
    seq_len=128,
    lr_enc=1e-4,
    lr_head=5e-4,
    warmup_ratio=0.1,
    seed=42,
    freeze_bottom_layers=0,
    tau=0.2,
    lambda_supcon=1.0,
    use_source_multilevel=True
)
os.makedirs(CFG["outdir"], exist_ok=True)
os.makedirs(os.path.join(CFG["outdir"], "ckpt"), exist_ok=True)


In [39]:
def set_seed(seed: int = 42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def l2_normalize(x: np.ndarray, eps: float = 1e-12):
    n = np.linalg.norm(x, axis=1, keepdims=True)
    return x / np.maximum(n, eps)

def pool_mean(last_hidden_state, attention_mask):
    # mean pooling with mask
    mask = attention_mask.unsqueeze(-1).float()
    summed = torch.sum(last_hidden_state * mask, dim=1)
    denom = mask.sum(dim=1).clamp(min=1e-6)
    return summed / denom


In [40]:
class TextDataset(Dataset):
    def __init__(self, df: pd.DataFrame, tok, max_len: int):
        self.df = df.reset_index(drop=True)
        self.tok = tok
        self.max_len = max_len

    def __len__(self): return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        enc = self.tok(
            str(row["text"]),
            truncation=True, padding="max_length",
            max_length=self.max_len, return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(int(row["label"]), dtype=torch.long),
            "src": str(row.get("src", "")),
        }


In [41]:
class ProjectionHead(nn.Module):
    def __init__(self, d_in: int, d_hidden: int = 256, d_out: int = 384, p_drop: float = 0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_in, d_hidden),
            nn.GELU(),
            nn.Dropout(p_drop),
            nn.Linear(d_hidden, d_out),
        )
        self.cls = nn.Linear(d_out, 2)

    def forward(self, feat, return_feat: bool = False):
        z = self.net(feat)
        z = F.normalize(z, p=2, dim=1)
        if return_feat:
            return z
        logits = self.cls(z)
        return logits, z


In [42]:
def supcon_loss(z, y, src_list, tau: float = 0.2, use_source_multilevel: bool = True):
    # z: (B,D) L2-normalized
    sim = (z @ z.T) / tau               # (B,B)
    B = z.size(0)
    device = z.device

    labels = y.view(-1, 1)              # (B,1)
    mask_pos = (labels == labels.T)     # positives: same class
    mask_self = torch.eye(B, device=device, dtype=torch.bool)
    mask_pos = mask_pos & (~mask_self)

    # exclude self from denom
    sim = sim - 1e9 * mask_self.float()

    # positive weights
    pos_w = torch.ones_like(sim)
    if use_source_multilevel and isinstance(src_list, list):
        src_eq = torch.zeros_like(sim, dtype=torch.bool)
        for i in range(B):
            si = src_list[i]
            for j in range(B):
                if i != j and si == src_list[j]:
                    src_eq[i, j] = True
        both_ai = (labels == 1) & (labels.T == 1)
        boost = (src_eq & both_ai)
        pos_w = pos_w + 0.5 * boost.float()  # +50% on same-source AI pairs

    log_prob = sim - torch.logsumexp(sim, dim=1, keepdim=True)
    loss_mat = -log_prob * mask_pos.float() * pos_w
    denom = mask_pos.float().sum(dim=1).clamp(min=1.0)
    loss_per_anchor = loss_mat.sum(dim=1) / denom
    return loss_per_anchor.mean()

def compute_metrics(y_true: np.ndarray, y_pred: np.ndarray):
    y_true = y_true.astype(int); y_pred = y_pred.astype(int)
    acc = float((y_true == y_pred).mean())

    def prf_for(label):
        tp = int(((y_pred == label) & (y_true == label)).sum())
        fp = int(((y_pred == label) & (y_true != label)).sum())
        fn = int(((y_pred != label) & (y_true == label)).sum())
        precision = tp / (tp + fp) if (tp + fp) else 0.0
        recall    = tp / (tp + fn) if (tp + fn) else 0.0
        f1        = 2*precision*recall / (precision + recall) if (precision + recall) else 0.0
        return precision, recall, f1

    p1, r1, f1 = prf_for(1)
    _, r0, _   = prf_for(0)
    avgrec = (r0 + r1) / 2.0
    return {"accuracy": acc, "precision_ai(1)": p1, "recall_ai(1)": r1, "f1_ai(1)": f1,
            "recall_human(0)": r0, "avgrec": avgrec}


In [43]:
set_seed(CFG["seed"])

df = pd.read_csv(CFG["csv_path"])
print("Columns:", list(df.columns))
for col in ("text", "label", "src"):
    assert col in df.columns, f"Missing required column: {col}"

df["text"] = df["text"].astype(str).str.strip()
df["src"]  = df["src"].fillna("").astype(str)
print("Rows:", len(df))
df.head(3)


Columns: ['text', 'label', 'src']
Rows: 10000


,text,label,src
0,Manning finished the year with a career-low 67...,0,squad_machine_continuation_30B
1,Let's not forget the traditional argument with...,0,xsum_machine_continuation_t0_3b
2,[title] Test dye a small patch of your running...,1,hswag_human


In [44]:
def stratified_split(indices, labels, test_size=0.2, seed=42):
    rng = np.random.default_rng(seed)
    indices = np.array(indices); labels = np.array(labels)
    train_idx, val_idx = [], []
    for lab in np.unique(labels):
        lab_idx = indices[labels == lab]
        rng.shuffle(lab_idx)
        n_val = max(1, int(round(len(lab_idx) * test_size)))
        val_idx.extend(lab_idx[:n_val].tolist())
        train_idx.extend(lab_idx[n_val:].tolist())
    rng.shuffle(train_idx); rng.shuffle(val_idx)
    return np.array(train_idx), np.array(val_idx)

idx_all = np.arange(len(df))
train_idx, val_idx = stratified_split(idx_all, df["label"].to_numpy(), test_size=0.2, seed=CFG["seed"])
len(train_idx), len(val_idx)


(8000, 2000)

In [45]:
tok = AutoTokenizer.from_pretrained(CFG["model_name"])
enc = AutoModel.from_pretrained(CFG["model_name"]).to(device)

if CFG["freeze_bottom_layers"] > 0 and hasattr(enc, "encoder") and hasattr(enc.encoder, "layer"):
    n = len(enc.encoder.layer)
    k = max(0, min(CFG["freeze_bottom_layers"], n))
    for i in range(k):
        for p in enc.encoder.layer[i].parameters():
            p.requires_grad = False
    print(f"Froze bottom {k} / {n} transformer blocks.")

head = ProjectionHead(d_in=enc.config.hidden_size, d_hidden=256, d_out=384, p_drop=0.1).to(device)


In [46]:
train_ds = TextDataset(df.iloc[train_idx], tok, CFG["seq_len"])
val_ds   = TextDataset(df.iloc[val_idx],   tok, CFG["seq_len"])

train_loader = DataLoader(train_ds, batch_size=CFG["batch_size"], shuffle=True,  num_workers=0, pin_memory=pin)
val_loader   = DataLoader(val_ds,   batch_size=CFG["batch_size"], shuffle=False, num_workers=0, pin_memory=pin)

probe = next(iter(DataLoader(TextDataset(df.iloc[:4], tok, CFG["seq_len"]),
                             batch_size=2, shuffle=False, num_workers=0)))
print({k: (type(v), getattr(v, 'shape', None)) for k,v in probe.items()})
print("src example:", probe["src"])


{'input_ids': (<class 'torch.Tensor'>, torch.Size([2, 128])), 'attention_mask': (<class 'torch.Tensor'>, torch.Size([2, 128])), 'labels': (<class 'torch.Tensor'>, torch.Size([2])), 'src': (<class 'list'>, None)}
src example: ['squad_machine_continuation_30B', 'xsum_machine_continuation_t0_3b']


In [47]:
enc_params  = [p for p in enc.parameters() if p.requires_grad]
head_params = list(head.parameters())
opt = torch.optim.AdamW([
    {"params": enc_params,  "lr": CFG["lr_enc"]},
    {"params": head_params, "lr": CFG["lr_head"]},
], weight_decay=1e-4)

total_steps = CFG["epochs"] * max(1, len(train_loader))
warmup_steps = int(CFG["warmup_ratio"] * total_steps)
sched = get_linear_schedule_with_warmup(opt, warmup_steps, total_steps)

# GradScaler: try new API, fallback to old if needed
try:
    scaler = torch.amp.GradScaler('cuda', enabled=(device == "cuda"))
except TypeError:
    scaler = torch.cuda.amp.GradScaler(enabled=(device == "cuda"))


In [48]:
best_sel = -1.0
best_state = None

for ep in tqdm(range(1, CFG["epochs"]+1), desc="Epochs", leave=True):
    # ---- Train ----
    enc.train(); head.train()
    tr_loss, n_tr = 0.0, 0
    pbar = tqdm(train_loader, desc=f"Train {ep}/{CFG['epochs']}", leave=False)
    for batch in pbar:
        labels = batch["labels"].to(device)
        sources = batch.get("src", [""] * labels.size(0))
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        opt.zero_grad(set_to_none=True)
        with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(device == "cuda")):
            out  = enc(input_ids=input_ids, attention_mask=attention_mask)
            feat = pool_mean(out.last_hidden_state, attention_mask)
            logits, z = head(feat)
            ce  = F.cross_entropy(logits, labels)
            scl = supcon_loss(z, labels, sources, tau=CFG["tau"], use_source_multilevel=CFG["use_source_multilevel"])
            loss = ce + CFG["lambda_supcon"] * scl

        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()
        sched.step()

        tr_loss += loss.item() * labels.size(0)
        n_tr    += labels.size(0)
        pbar.set_postfix(loss=f"{loss.item():.4f}", ce=f"{ce.item():.4f}", scl=f"{scl.item():.4f}")

    # ---- Validate ----
    enc.eval(); head.eval()
    y_true, y_pred, vl_loss, n_vl = [], [], 0.0, 0
    pbar_val = tqdm(val_loader, desc=f"Val   {ep}/{CFG['epochs']}", leave=False)
    with torch.no_grad():
        for batch in pbar_val:
            labels = batch["labels"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(device == "cuda")):
                out  = enc(input_ids=input_ids, attention_mask=attention_mask)
                feat = pool_mean(out.last_hidden_state, attention_mask)
                logits, z = head(feat)
                ce  = F.cross_entropy(logits, labels)
            preds = torch.argmax(logits, dim=1)
            y_true.append(labels.cpu().numpy())
            y_pred.append(preds.cpu().numpy())
            vl_loss += ce.item() * labels.size(0)
            n_vl    += labels.size(0)

    y_true = np.concatenate(y_true); y_pred = np.concatenate(y_pred)
    mets = compute_metrics(y_true, y_pred)
    avg_tr = tr_loss / max(1, n_tr)
    avg_vl = vl_loss / max(1, n_vl)
    print(f"[Epoch {ep}] train_loss={avg_tr:.4f}  val_loss={avg_vl:.4f}  "
          f"Acc={mets['accuracy']:.4f}  AvgRec={mets['avgrec']:.4f}  F1_AI={mets['f1_ai(1)']:.4f}")

    # Select by balanced recall
    if mets["avgrec"] > best_sel:
        best_sel = mets["avgrec"]
        best_state = {
            "encoder": enc.state_dict(),
            "head": head.state_dict(),
            "val_metrics": mets,
            "config": CFG
        }

# Save best
ckpt_path = os.path.join(CFG["outdir"], "ckpt", "best.pt")
torch.save(best_state, ckpt_path)
print("Saved checkpoint:", ckpt_path, "  (AvgRec best:", f"{best_sel:.4f})")


Epochs:   0%|          | 0/6 [00:00<?, ?it/s]

Train 1/6:   0%|          | 0/63 [00:00<?, ?it/s]

Val   1/6:   0%|          | 0/16 [00:00<?, ?it/s]

[Epoch 1] train_loss=5.6050  val_loss=0.6176  Acc=0.6470  AvgRec=0.5000  F1_AI=0.0000


Train 2/6:   0%|          | 0/63 [00:00<?, ?it/s]

...

Val   5/6:   0%|          | 0/16 [00:00<?, ?it/s]

[Epoch 5] train_loss=4.5407  val_loss=0.4301  Acc=0.8230  AvgRec=0.7828  F1_AI=0.7204


Train 6/6:   0%|          | 0/63 [00:00<?, ?it/s]

Val   6/6:   0%|          | 0/16 [00:00<?, ?it/s]

[Epoch 6] train_loss=4.4329  val_loss=0.4715  Acc=0.8175  AvgRec=0.7701  F1_AI=0.7020
Saved checkpoint: data\ckpt\best.pt   (AvgRec best: 0.7828)


In [49]:
# Reload best (just to be sure)
state = torch.load(ckpt_path, map_location=device)
enc.load_state_dict(state["encoder"]); head.load_state_dict(state["head"])
enc.eval(); head.eval()

df_eval = pd.read_csv("data/data.csv")

full_loader = DataLoader(TextDataset(df_eval, tok, CFG["seq_len"]),
                         batch_size=CFG["batch_size"], shuffle=False, num_workers=0, pin_memory=pin)

all_chunks = []
for batch in tqdm(full_loader, desc="Export 384-d", leave=True):
    with torch.no_grad(), torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(device == "cuda")):
        out  = enc(input_ids=batch["input_ids"].to(device),
                   attention_mask=batch["attention_mask"].to(device))
        feat = pool_mean(out.last_hidden_state, batch["attention_mask"].to(device))
        z    = head(feat, return_feat=True)  # (B,384), already L2-normalized
    all_chunks.append(z.cpu().numpy())

Z = np.vstack(all_chunks)
Z = l2_normalize(Z)  # idempotent but safe
np.save(os.path.join(CFG["outdir"], "detective_emb_384.npy"), Z)
np.save(os.path.join(CFG["outdir"], "train_idx.npy"), train_idx)
np.save(os.path.join(CFG["outdir"], "val_idx.npy"),   val_idx)

with open(os.path.join(CFG["outdir"], "Detective_report.json"), "w") as f:
    json.dump({"best_AvgRec": float(best_sel),
               "val_metrics": state["val_metrics"],
               "ckpt": ckpt_path}, f, indent=2)

print("Embeddings saved to:", os.path.join(CFG["outdir"], "text_embeddings_detective.npy"))


C:\Users\Steven\AppData\Local\Temp\ipykernel_4860\2196016986.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(ckpt_path, map_location=device)


Export 384-d:   0%|          | 0/79 [00:00<?, ?it/s]

Embeddings saved to: data\text_embeddings_detective.npy
